In [1]:
# !pip install datasets
# !pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
from datasets import load_dataset
from transformers import pipeline
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk

In [3]:
nltk.download("punkt")
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [36]:
#Завантажуємо текст новин Володимира Зеленського
dataset = load_dataset("slava-medvedev/zelensky-speeches")

# Беремо 5 рандомних текстів
num_samples = 5
samples = dataset['train'].select(range(num_samples))

In [45]:
model = pipeline("summarization", model="yelyah/mT5-XLSUM-ua-news") #Fine-tuned модель яка навчена на укр.новинах

config.json:   0%|          | 0.00/937 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

Device set to use cuda:0


In [46]:
samples[0]

{'date': 1637485200,
 'link': 'https://www.president.gov.ua/news/zvernennya-prezidenta-ukrayini-z-nagodi-dnya-gidnosti-ta-svo-71637',
 'topic': 'Звернення Президента України з нагоди Дня Гідності та Свободи',
 'full_text': "Шановні громадяни України! На нашій планеті живе близько п’яти тисяч народів. Їхній портрет, архетип, ментальність – це заплутана павутина історичних подій для етнографів. Але є одна річ, яка, на мій погляд, просто, але досить точно змальовує характер народу. Це його державні свята. Те, що об’єднує всіх, тож є важливим для всіх. 21 листопада – саме такий день. Відповідь на питання: хто такі українці та що для них головне? Ми відзначаємо сьогодні День Гідності та Свободи. Свято, яке показує, що для нас гідність і свобода – це свято. Це наше повітря. Ні. Це не звучить занадто гучно. Коли хтось вирішує перекрити нам повітря – ось тоді буває занадто гучно. Коли влада не чує український народ – тоді буває занадто гучно. Тому ми вільні. Вільні творити майбутнє. І маємо пи

In [47]:
# Ініціалізація ROUGE-скора
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=False)
smooth = SmoothingFunction().method1

def evaluate_summaries(samples):
    bleu_scores = []
    meteor_scores = []
    rouge_1, rouge_2, rouge_l = [], [], []

    for sample in samples:
        reference = sample["topic"]  # Саммарі текст
        article = sample["full_text"]  # Оригінальний текст

        #generated summary
        generated = model(article[:1024], max_length=150, min_length=50, do_sample=False)[0]["summary_text"]

        # Токенізація тексту для METEOR
        reference_tokens = word_tokenize(reference)
        generated_tokens = word_tokenize(generated)

        # BLEU Score
        bleu = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smooth)
        bleu_scores.append(bleu)

        # METEOR Score
        meteor = meteor_score([reference_tokens], generated_tokens)
        meteor_scores.append(meteor)

        # ROUGE Score
        reference = " ".join(reference.split())  # Прибрати зайві пробіли
        generated = " ".join(generated.split())
        rouge = scorer.score(reference, generated)
        
        rouge_1.append(rouge["rouge1"].fmeasure)
        rouge_2.append(rouge["rouge2"].fmeasure)
        rouge_l.append(rouge["rougeL"].fmeasure)

        print("REFERENCE:", reference)
        print("GENERATED:", generated)

    return {
        "BLEU": sum(bleu_scores) / num_samples,
        "METEOR": sum(meteor_scores) / num_samples,
        "ROUGE-1": sum(rouge_1) / num_samples,
        "ROUGE-2": sum(rouge_2) / num_samples,
        "ROUGE-L": sum(rouge_l) / num_samples
    }

In [48]:
metrics = evaluate_summaries(samples)
print(metrics)

REFERENCE: Звернення Президента України з нагоди Дня Гідності та Свободи
GENERATED: День Гідності і Свободи: хто такі українці та що для них головне? Наталка Яресько, автор книги "День гідності та свободи", яка проходить 21 листопада
REFERENCE: Звернення Президента України з нагоди Дня пам'яті жертв голодоморів
GENERATED: Дорогий народе! День пам’яті жертв голодоморів для багатьох із нас насправді перша. Що це означає і навіщо ми вшановуємо загиблих українців
REFERENCE: Звернення Президента до світового українства
GENERATED: Сьогодні вшановують пам’ять мільйонів українців, знищених під час жахливого Голодомору 1932-1933 років і 1945-го року - День пам'яті жертв голодоморів
REFERENCE: Виступ Президента Володимира Зеленського у Верховній Раді зі щорічним Посланням про внутрішнє і зовнішнє становище України
GENERATED: Дорогий український народе: Послання про незалежність варто змінити і вітаючи всіх їх у цій залі - Володимир Зеленський, а також міністр оборони Арсен Аваков
REFERENCE: Звер